# LOGISTIC REGRESSION

#### LOAD THE DEPENDANCIES

In [241]:
import pandas as pd
from pandas import set_option
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import set_printoptions

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import os
import sys
sys.path.insert(0, "C:\\Users\\Crystal\\Desktop\\Programs\\my-modules-and-libraries")

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from scipy import stats
import seaborn as sns
import math



In [242]:
#### KNN Classifier

def logistic_regression(X_train,y_train,X_test,y_test):
    """Logistic Regression algorithm"""
    global classifier
    
    f1_scores=[]
    accur=[]
    preci=[]
    recall=[]
    
        
    # Define Logistic Regression Model
    classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, 
                                    C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, 
                                    random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', 
                                    verbose=0, warm_start=False, n_jobs=None)

    # Fit Model
    classifier.fit(X_train,y_train)

    y_pred=classifier.predict(X_test)

    y_prob=classifier.predict_proba(X_test)

    y_prob=y_prob[:,1]

    f1,a,p,r=metrics(y_test, y_pred)

    f1_scores.append(f1)
    accur.append(a)
    preci.append(p)
    recall.append(r)
        
    print('\n','f1_scores: ',f1_scores)
    print('accuracy: ',accur)
    
    return f1_scores,accur,preci,recall,y_pred,y_prob
        
        

In [243]:
#### Evaluate Model

def metrics(y_test, y_pred):
    """Confusion matrix and associated metrics"""
    matrix = confusion_matrix(y_test, y_pred)
#     print(y_test.shape)

    y_test=y_test.values.reshape(y_test.size)
#     print(y_test.shape)
#     print(y_pred.shape)
    
    tn,fp,fn,tp=confusion_matrix(y_test,y_pred).ravel()
    precision=precision_score(y_test,y_pred)
    recall=recall_score(y_test,y_pred)
    f1=f1_score(y_test,y_pred)
    accuracy=accuracy_score(y_test,y_pred)
    print('Confusion matrix breakdown:',('tn:',tn,'fp:',fp,'fn:',fn,'tp:',tp),'\n')
    print('Confusion matrix:\n', matrix)
    print('Precision: When it predicts yes, how often is it correct?:',precision)
    print('Recall.True Positive Rate: When it\'s actually yes, how often does it predict yes?:',recall)
    print('F1:score is the harmonic average of the precision and recall,:',f1)
    print('Accuracy.Overall, how often is the classifier correct?: ',accuracy)
    print('Misclassification Rate.Overall, how often is it wrong?: ',(1-accuracy))
    
    plot_confusion_matrix(matrix)

    return (f1,accuracy,precision,recall)
    


In [244]:
def plot_confusion_matrix(cm,classes):
    fig, ax = plt.subplots()
    cmap=plt.cm.Blues
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(cm.shape[1]),
    yticks=np.arange(cm.shape[0]),
    xticklabels=classes,
    yticklabels=classes,
    title="confusion",
    ylabel='True label',
    xlabel='Predicted label')

    # Loop over data dimensions and create text annotations.
    normalize=False
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
            ha="center", va="center",
            color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()



    plt.show()

#### PREPROCESSING: Replacing zeros where it is not a valid value for that feature.
##### This done here by replacing the zero values with a NAN, then replacing the NAN with the average value for non-zero values in that column

In [245]:
def replacing_zeros(dataset,the_headers):
    """Function used to remove zeros from numeric features when 0 is not practical"""

    for header in the_headers:
        dataset[header]=dataset[header].replace(0,np.nan)
        mean=int(dataset[header].mean(skipna=True))
        dataset[header]=dataset[header].replace(np.nan,mean)
        
    return dataset


#### PREPROCESSING: Split the dataset

In [246]:
def split_the_dataset(dataset,input_headers,target_header):
    
#     X=dataset.iloc[:,0,8]
#     X=dataset.iloc[:,[1,2,4,5,6,7]]
#     y=dataset.iloc[:,8]
    X=dataset[input_headers]
    y=dataset[target_header]
    
    X.head()
    
    return X,y

#### PREPROCESSING: Quick look at the features

In [247]:
def quick_feature_view(X):
    

    # X.hist(bins=50,figsize=(15,15))
    # X.plot(kind='hist',subplots=True,layout=(3,3),sharex=False, figsize=(15,15))

    headers=X.columns.tolist()
    fig, axes = plt.subplots(nrows=1, ncols=len(headers), figsize=(20, 10))
    print(headers)
    for i,head in enumerate(headers,0):

        axes[i].hist(x=X[head],bins=50,edgecolor='black')
        axes[i].set(title=head)
        axes[i].grid()

    plt.show()

    X.plot(kind='density',subplots=True,layout=(3,3),sharex=False, figsize=(15,15))
    plt.show()

#### PREPROCESSING:Target Summary

In [248]:
def target_summary(dataset,target_header):
    """PREPROCESSING:Target Summary"""
    print(dataset.groupby(target_header).size())
    print((dataset.groupby(target_header).size()/len(y)*100))


#### PREPROCESSING:Train - Test Split of the data

In [249]:
def split_the_train_test_data(X,y):
    
    """PREPROCESSING:Train - Test Split of the data"""
    X_train,X_test,y_train,y_test=train_test_split(X, y,test_size=.30,random_state=42,shuffle=True)
    X_train.head()
    
    return X_train,X_test,y_train,y_test

#### PREPROCESSING: Feature Scaling

In [250]:
def feature_scaling(X_train,X_test):
    sc_X=StandardScaler()
    X_train=sc_X.fit_transform(X=X_train,y=None)
    X_test=sc_X.fit_transform(X=X_test,y=None)

    print(sc_X.fit(X_train))
    print(X_train[0:5])
    
    return X_train, X_test



In [251]:
def plot_the_metrics(f1_scores,accur,preci,recall):

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10),sharex='none')
    axes[0,0].plot(k,f1_scores,marker='o')
    axes[0,1].plot(k,accur,marker='o')
    axes[0,0].set(title='F1 Score')
    axes[0,1].set(title='Accuracy')
    axes[0,0].set(xlabel='K value')
    axes[0,1].set(xlabel='K value')
#     axes[0,0].set(xlim=(3,21), ylim=(0,1))
#     axes[0,1].set(xlim=(3,21), ylim=(0,1))
    axes[0,0].set(xticks=range(3,23,2),yticks=np.arange(0,1.1,.1))
    axes[0,1].set(xticks=range(3,23,2),yticks=np.arange(0,1.1,.1))
    axes[0,0].grid()
    axes[0,1].grid()

    axes[1,0].plot(k,preci,marker='o')
    axes[1,1].plot(k,recall,marker='o')
    axes[1,0].set(title='Precision')
    axes[1,1].set(title='Recall')
    axes[1,0].set(xlabel='K value')
    axes[1,1].set(xlabel='K value')
#     axes[1,0].set(xlim=(3,21), ylim=(0,1))
#     axes[1,1].set(xlim=(3,21), ylim=(0,1))
    axes[1,0].set(xticks=range(3,23,2),yticks=np.arange(0,1.1,.1))
    axes[1,1].set(xticks=range(3,23,2),yticks=np.arange(0,1.1,.1))
    axes[1,0].grid()
    axes[1,1].grid()

    plt.show()

In [252]:
def box_plot_the_metrics(f1_scores,accur,preci,recall):
    """Box plots for the classification metrics over a range of parameter adjustments"""
    
    f=np.asarray(f1_scores)
    a=np.asarray(accur)
    p=np.asarray(preci)
    r=np.asarray(recall)
    
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))
    axes.boxplot([f,a,p,r])
    axes.set_xticklabels(['f1_score','accuracy','precision','recall'])
    plt.grid()
    
    plt.show()
    

In [253]:
def plot_of_data_space(data,labels,input_headers):
    
    
    xx_1=pd.DataFrame(data[:,0]) 
    xx_2=pd.DataFrame(data[:,1]) 
    y=pd.DataFrame(labels)

#     print(xx_1.head(),'\n',xx_2.head(),'\n',y.head())
#     print(list(xx_1[y==0]))
#     print(list(xx_1[y==1]))
    plt.figure(figsize=(15,10)) 
    b=plt.scatter(xx_1[y==0],xx_2[y==0],color='b') 
    r=plt.scatter(xx_1[y==1],xx_2[y==1],color='r')
    g=plt.scatter(xx_1[y==2],xx_2[y==2],color='g')

#     plt.scatter(data[:,0],data[:,1],s=40,c=labels,cmap=plt.cm.Spectral)
#     plt.scatter(x_test[:,0],x_test[:,1],color='black')

    plt.xlabel(input_headers[0])
    plt.ylabel(input_headers[1])
#     plt.xlim(xx_1.min(),xx_1.max())
#     plt.ylim(xx_2.min(),xx_2.max())
    plt.grid()
    plt.legend((b,r,g),tuple(np.unique(labels)))
    plt.show()


In [254]:
def boundary_decision_plot(X,y,X_train,y_train,x_test,y_pred,y_prob):
    
    X_unscaled=X.values
    X_scaled, dummy=feature_scaling(X_unscaled,X_test=1)
    xx_1=pd.DataFrame(X_train[:,0]) 
    xx_2=pd.DataFrame(X_train[:,1]) 
    y=pd.DataFrame(y_train.values)
    
#     print(y_train[0:5])
#     print(X_train[0:5])

    xx_test_1=pd.DataFrame(x_test[:,0]) 
    xx_test_2=pd.DataFrame(x_test[:,1])

    y_predict=pd.DataFrame(y_pred) 
    y_prob=pd.DataFrame(y_prob) 

    cmap_light = ListedColormap(['#FFAAAA','#AAAAFF'])

#     cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
    cmap_bold = ListedColormap(['#FF0000','#0000FF'])

    h=.02

#     Plot the decision boundary. For that, we will assign a color to each point in the mesh [x_min, x_max]x[y_min, y_max].
#     x1=X_train[:,0] 
    x1=X_scaled[:,0]
    x2=X_scaled[:,1] 
    x_min,x_max = x1.min()-1,x1.max()+1 
    y_min,y_max = x2.min()-1,x2.max()+1 
    xx,yy = np.meshgrid(np.arange(x_min,x_max,h),np.arange(y_min,y_max,h))


    the_predict=classifier.predict(np.c_[xx.ravel(),yy.ravel()])

#     Put the result into a color plot
    Z = the_predict.reshape(xx.shape) 
    plt.figure(figsize=(15,15)) 
    plt.xlim(xx.min(),xx.max()) 
    plt.ylim(yy.min(),yy.max())

    plt.pcolormesh(xx,yy,Z,cmap=cmap_light)

#     plt.scatter(xx_1[y==0],xx_2[y==0],color='b',marker='o') 
#     plt.scatter(xx_1[y==1],xx_2[y==1],color='r',marker='o')
    

#     plt.scatter(X_train[:,0],X_train[:,1],s=40,c=y_train,cmap=plt.cm.Spectral)
    cm=plt.cm.get_cmap('RdYlBu_r')

#     plt.scatter(xx_test_1[y_predict==0],xx_test_2[y_predict==0],cmap=cm,vmin=0,vmax=1,c=y_predict,marker='D') 
#     plt.scatter(xx_test_1[y_predict==1],xx_test_2[y_predict==1],cmap=cm,vmin=0,vmax=1,c=y_predict,marker='D') 
    plt.scatter(xx_test_1[y_predict==0],xx_test_2[y_predict==0],cmap=cm,vmin=0,vmax=1,c=y_prob,marker='D') 
    plt.scatter(xx_test_1[y_predict==1],xx_test_2[y_predict==1],cmap=cm,vmin=0,vmax=1,c=y_prob,marker='D') 
    plt.colorbar()


    plt.grid() 
    plt.show()


In [255]:
def max_results(f1_scores,accur,preci,recall):
    
    f=np.asarray(f1_scores)
    a=np.asarray(accur)
    p=np.asarray(preci)
    r=np.asarray(recall)
    
    
    results=f"""\n
    The max F1 SCORE is {round((f.max()*100),1)}% 
    The max ACCURACY is {round((a.max()*100),1)}% 
    The max PRECISION is {round((p.max()*100),1)}% 
    The max RECALL is {round((r.max()*100),1)}%
    """
    
    print(results)
    
    
    
    
    

In [256]:
def label_encoding(dataset,input_headers):
    
    for i in input_headers:
        
        the_data_type=dataset[i].dtype.name
        if (the_data_type=='object'):
            lable_enc=preprocessing.LabelEncoder()
            lable_enc.fit(dataset[i])
            labels=lable_enc.classes_   #this is an array
            labels=list(labels) #converting the labels array to a list
            print(labels)
            dataset[i]=lable_enc.transform(dataset[i])
#             print(dataset[i])

    return dataset[i],labels
    

In [257]:
def replace_zeros(dataset,header):
    # Replace zeros with the mean where needed.
    rz=input('Do you need to replace any zeros in the dataset?')
    if (rz.lower()=='y'):
        the_headers=[header]
        dataset=replacing_zeros(dataset,the_headers)
        dataset.head()
    

# MAIN PROGRAM

In [258]:
if __name__ == "__main__":
    
    global classifier
    
#     RETRIEVE THE DATASET
    
    location=r'C:/Users/Crystal/Desktop/Programs/dataset_repo/ml_NF/ml_NFL_pass_run.csv'
#     location=r'C:\Users\Crystal\Desktop\Programs\dataset_repo\diabetes.csv'
#     location=r'C:\Users\Crystal\Desktop\Programs\dataset_repo\CDH_Train.csv'
    dataset=pd.read_csv(location)
#     dataset=pd.read_pickle(location)

    print(dataset.info())
    dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7109 entries, 0 to 7108
Data columns (total 43 columns):
play_id                      7109 non-null int64
game_id                      7109 non-null int64
home_team                    7109 non-null object
away_team                    7109 non-null object
posteam                      7109 non-null object
posteam_type                 7109 non-null object
defteam                      7109 non-null object
side_of_field                7109 non-null object
yardline_100                 7109 non-null float64
game_date                    7109 non-null object
quarter_seconds_remaining    7109 non-null float64
half_seconds_remaining       7109 non-null float64
game_seconds_remaining       7109 non-null float64
game_half                    7109 non-null object
quarter_end                  7109 non-null int64
drive                        7109 non-null int64
sp                           7109 non-null int64
qtr                          7109 non-null i

In [259]:
#     replace_zeros(dataset,header)


### Selecting inputs and targets

In [260]:
    #Selecting inputs and targets
    
    target_header=['play_type']
    input_headers=['score_differential','game_seconds_remaining','yardline_100','qtr']
    
    target_label=label_encoding(dataset,target_header)
    
#     print('target label:',target_label)
    test_label=label_encoding(dataset,input_headers)
#     print('test label:',test_label)
    X,y=split_the_dataset(dataset,input_headers,target_header)
    if (X.values.shape[1]==2):
#         print(y.values)
        plot_of_data_space(X.values,y.values,input_headers)
        
#     print(X.head())

['pass', 'run']


UnboundLocalError: local variable 'labels' referenced before assignment

In [237]:
target_label

0       0
1       1
2       0
3       1
4       0
5       1
6       0
7       0
8       1
9       0
10      0
11      1
12      0
13      1
14      0
15      0
16      1
17      1
18      0
19      0
20      0
21      1
22      0
23      0
24      0
25      1
26      1
27      0
28      1
29      0
       ..
7079    0
7080    0
7081    1
7082    1
7083    0
7084    0
7085    0
7086    1
7087    1
7088    0
7089    0
7090    0
7091    1
7092    0
7093    0
7094    1
7095    1
7096    1
7097    0
7098    1
7099    1
7100    0
7101    0
7102    1
7103    0
7104    0
7105    1
7106    0
7107    1
7108    1
Name: play_type, Length: 7109, dtype: int64

In [238]:
    #Splitting the Train-Test data
    X_train,X_test,y_train,y_test=split_the_train_test_data(X,y)
    y_test.shape

    

(2133, 1)

In [239]:
    #Scale the data    
    X_train, X_test=feature_scaling(X_train,X_test)

StandardScaler(copy=True, with_mean=True, with_std=True)
[[ 0.11263205 -0.70142389 -1.26393251  0.40055421]
 [ 0.30842492 -1.15008369  0.409369    1.28248243]
 [ 0.11263205  1.04283616 -1.67205483 -1.36330222]
 [-0.572643   -0.45998408 -0.65174903  0.40055421]
 [ 0.60211423 -1.18145185  0.85830355  1.28248243]]


In [240]:
    #Call the Logistic Regression function
    f1_scores,accur,preci,recall,y_pred,y_prob=logistic_regression(X_train,y_train,X_test,y_test)
    

Confusion matrix breakdown: ('tn:', 1014, 'fp:', 219, 'fn:', 654, 'tp:', 246) 

Confusion matrix:
 [[1014  219]
 [ 654  246]]
Precision: When it predicts yes, how often is it correct?: 0.5290322580645161
Recall.True Positive Rate: When it's actually yes, how often does it predict yes?: 0.2733333333333333
F1:score is the harmonic average of the precision and recall,: 0.3604395604395604
Accuracy.Overall, how often is the classifier correct?:  0.5907172995780591
Misclassification Rate.Overall, how often is it wrong?:  0.4092827004219409


C:\Users\Public\Programming\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Public\Programming\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = None.
  " = {}.".format(self.n_jobs))


TypeError: plot_confusion_matrix() missing 1 required positional argument: 'classes'

In [ ]:
    #Plot the metrics
#     plot_the_metrics(f1_scores,accur,preci,recall)
    

In [ ]:
    #Box plot of the metrics
    box_plot_the_metrics(f1_scores,accur,preci,recall)

In [ ]:
    max_results(f1_scores,accur,preci,recall)

In [ ]:
    #Plot the decision boundaries (for input vectors only)
    
    if (X.values.shape[1]==2):
        boundary_decision_plot(X,y,X_train,y_train,X_test,y_pred,y_prob)

In [ ]:
dataset.shape

In [ ]:
dataset[dataset.down.isna()].play_type